In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
import numpy as np
import pickle

# Load dataset
df = pd.read_csv("new.csv")

# Features and target
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Calculate scale_pos_weight
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

# Train XGBoost with class weight
model = xgb.XGBClassifier(
    max_depth=3,
    learning_rate=0.1,
    n_estimators=100,
    objective='binary:logistic',
    scale_pos_weight=scale_pos_weight,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

model.fit(X_train_scaled, y_train)

# Predict probabilities instead of hard labels
y_probs = model.predict_proba(X_test_scaled)[:, 1]

# Use threshold (you can tune this)
threshold = 0.35
y_pred = (y_probs >= threshold).astype(int)

# Evaluate
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Save the model
with open('xgboost_model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model saved successfully as 'xgboost_model.pkl'")

# Also save the scaler for future use
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

print("Scaler saved successfully as 'scaler.pkl'")

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.88      0.92       246
           1       0.35      0.67      0.46        24

    accuracy                           0.86       270
   macro avg       0.66      0.77      0.69       270
weighted avg       0.91      0.86      0.88       270

Confusion Matrix:
 [[216  30]
 [  8  16]]
Model saved successfully as 'xgboost_model.pkl'
Scaler saved successfully as 'scaler.pkl'


C:\Users\shrin\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:183: UserWarning: [20:32:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
